# 5.4 PyTorch常用工具介绍

## 5.4.1 优化器

优化器（optimzier）是指根据神经网络反向传播的梯度信息来自动更新网络模型的参数，以起到降低loss函数计算值的作用。PyTorch将深度学习中的常用参数更新优化方法全部封装在torch.optim包中。

通常，我们直接使用优化器完成自动参数更新，使模型更好的收敛，获得更好的训练效果。

In [13]:
opt = optim.SGD(model.parameters(), lr = lr)  # SGD优化器实例化
# momentum动量加速，在SGD函数里指定momentum的值即可
opt_Momentum = optim.SGD(model.parameters(), lr=lr, momentum=0.8)
# RMSprop, 需要设置超参数alpha
opt_RMSprop = optim.RMSprop(model.parameters(), lr=lr, alpha=0.9) 
# Adam，设置参数betas=(0.9, 0.99)
opt_Adam = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.99))

## 5.4.2 Dataset和DataLoader

（1）创建和使用dataset

在使用torch.utils.data.datset抽象类之前，首先要导入该类：


In [14]:
import torch.utils.data.dataset as Dataset

在应用Dataset抽象类创建子类时，通常需要重写__ inti __ 初始化方法来定义数据内容和标签，重写 __ len __ 方法来返回数据集大小，以及重写 __ getitem __ 方法来得到数据内容和标签。

下面是一个简单的创建子类的示例。


In [15]:
import torch
import torch.utils.data.dataset as Dataset
import numpy as np

#创建子类
class subDataset(Dataset.Dataset):
    #初始化，定义数据内容和标签
    def __init__(self, Data, Label):
        self.Data = Data
        self.Label = Label
    #返回数据集大小
    def __len__(self):
        return len(self.Data)
    #得到数据内容和标签
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = torch.Tensor(self.Label[index])
        return data, label

# 构建数据集
Data = np.asarray([[3, 2], [1, 4], [7, 2], [3, 1]])
Label = np.asarray([[0], [0], [1], [1]])

if __name__ == '__main__':
    sub = subDataset(Data, Label)  # 创建数据集对象
    print('数据集大小为：', sub.__len__())  # 获取dataset数据的大小
    print(sub.__getitem__(0))  # 第0项的数据
    print(sub[0])  # 效果等同于__getitem__

数据集大小为： 4
(tensor([3., 2.]), tensor([0.]))
(tensor([3., 2.]), tensor([0.]))


以上示例中，用户通过自己创建的Dataset对象的子类来构建数据集对象，之后就可以使用该对象来读取数据集的相关数据。

（2）创建和使用DataLoader

DataLoader类提供了通过用户构建的数据集对象读取数据的方法。在使用该类时，首先进行导入：


In [19]:
import torch.utils.data.dataloader as DataLoader

在创建Dataloader迭代器对象时，需将将用户构建的数据集对象作为参数。例如创建要给DataLoader对象，该对象对应的数据集对象为sub，读取数据时，设置batchsize为2，表示每批处理两个数据，shuffle为false表示不打乱数据的顺序，num_workers=1表示使用1个子进程来处理加载数据，定义对象代码如下：

In [22]:
dataloader = DataLoader.DataLoader(sub, batch_size=2,shuffle=False,num_workers=0)

之后就可以使用如下代码进行批量读取数据：

In [23]:
for i, item in enumerate(dataloader):
    data, label = item
    print('data:', data)
    print('label:', label)

data: tensor([[3., 2.],
        [1., 4.]])
label: tensor([[0.],
        [0.]])
data: tensor([[7., 2.],
        [3., 1.]])
label: tensor([[1.],
        [1.]])


通过执行结果可以看到，每次读取到的数据是多个样本及其标签，样本数量是通过batch_size设定的。在实际训练或推理过程中，可以根据计算环境（如CPU、GPU）及每个样本的大小来选择合适的batch_size，使网络尽量可以以较大吞吐量并行快速计算，从而加快训练或推理的速度。

## 5.4.3 torchvision

torchvision包是PyTorch的一个扩展包，收录了若干重要的公开数据集，网络模型和常用的图像变换方法，以便于研究者进行图像处理和识别方法的实验和学习。

（1）torchvision.datasets数据集下载模块

使用torchvision.datasets数据集下载模块提供的工具可以下载许多公开的经典数据集用于实验，如mnist手写数据集、CIFAR10图像十分类数据集等，下面演示如何使用torchvision.datasets模块下载CIFAR10数据集。


In [24]:
import torch
import torch.utils.data.dataset as Dataset
import torchvision
# 全局取消证书验证,数据集更容易被下载成功
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# 分别下载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=None)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


（2）torchvision.models预训练模型模块

torchvision.models模块封装了常用的各种神经网络结构，如alexnet、densenet、inception、resnet、VGG等，并提供了相应的预训练模型，通过简单调用便可以来读取网络结构和预训练模型，以便针对实际任务进行模型微调。

以下代码将resnet50预训练模型加载：


In [26]:
import torchvision

model = torchvision.models.resnet50(pretrained=True)

在导入resnet50预训练模型时，设置pretrained=True表示使用预训练模型的参数进行初始化网络参数，否则不使用预训练模型的参数进行初始化。

以下代码将VGG16预训练模型加载，并打印其网格模型结构：


In [27]:
import torchvision.models as models
model = models.vgg16(pretrained=True)
print(model)

D:\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\86188/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

从模型的打印结果可以发现，最后的输出层有1000个神经元结点。在实际任务中，往往需要修改最后的输出层结点数，以便于使用新的训练样本针对新的任务进行训练。以下示例给出了修改上面的网络模型输出层为10个结点的示例。

In [28]:
model.classifier[6]=torch.nn.Linear(4096,10)
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)


通常，我们只需要对最后几层网络的参数进行学习，而将前面各层的预训练好的参数固定下来。下面的示例给出了上述模型除最后三层的参数外其他所有参数固定下来的代码，即将要固定的参数的requires_grad的值为False。在训练时，优化器只需要对requires_grad的值为True的参数继续学习，代码如下：

In [29]:
from torch import optim as optimizer 
for p in model.parameters():
    p.requires_grad=False
for p in model.classifier.parameters():
    p.requires_grad=True
opt=optimizer.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

当然，也可以直接在定义优化器时直接给出需要学习的参数。因此，上述的代码也可以直接写成：

In [30]:
opt=optimizer.SGD(filter(lambda p: p.requires_grad, model.classifier.parameters()), lr=1e-3)

接下来，就可以使用新样本对模型进行训练了。在训练时，就只对后三层网络的参数进行学习，其他各层的参数都是预训练好的，主要用于特征提取。

（3）torchvision.transforms图像变换模块

torchvision.transforms模块提供的图像变换可以完成图像尺寸放缩、切割、翻转、边填充、归一化等操作，这些操作可以从原始图像中得到更多的图像，从而实现了图像增强，丰富了训练样本。

以下代码给出了图像的各种变换示例。

In [31]:
import torch
import torchvision
import PIL
orig_img1 = torch.randn(4, 3, 64, 64)
orig_img2 = torch.randn(3, 64, 64)
print(orig_img1.shape, orig_img2.shape)
# 对图像进行放缩，默认插值方法是线性插值
resize = torchvision.transforms.Resize((8, 8), interpolation=PIL.Image.BILINEAR)
new_img1 = resize(orig_img1)
new_img2 = resize(orig_img2)
print(new_img1.shape, new_img2.shape)
# 对图像进行中心切割
ccrop = torchvision.transforms.CenterCrop((16,16))  # 切割得到的尺寸为(16,16)
new_img3 = ccrop(orig_img1)
new_img4 = ccrop(orig_img2)
print(new_img3.shape, new_img4.shape)
# 对图像进行随机切割
rcrop = torchvision.transforms.RandomCrop(6)  # 切割得到的尺寸为(6,6)，注意参数为整数
new_img5 = rcrop(orig_img1)
new_img6 = rcrop(orig_img2)
print(new_img5.shape, new_img6.shape)
# 对图像先进行随机切割，然后再resize成给定的size大小
rrcrop = torchvision.transforms.RandomResizedCrop(2)
new_img7 = rrcrop(orig_img1)
new_img8 = rrcrop(orig_img2)
print(new_img7.shape, new_img8.shape)
# 对图像边缘进行扩充
pad = torchvision.transforms.Pad(padding=1, fill=0)  # 上下左右各扩充1行或1列值为0的像素
new_img9 = pad(new_img7)
new_img10 = pad(new_img8)
print(new_img9.shape, new_img10.shape)
# 对图像RGB各通道像素按正态分布进行归一化
norm = torchvision.transforms.Normalize((-1,1,0), (1,2,3))
new_img11 = norm(new_img7)
new_img12 = norm(new_img8)
print(f"{new_img8}, {new_img12}")
# 转换为Image图像
topil= torchvision.transforms.ToPILImage()
new_img14 = topil(new_img8)
print(new_img8.shape, new_img14.size)
new_img14.save("a.png")  # 将图像保存为文件

torch.Size([4, 3, 64, 64]) torch.Size([3, 64, 64])
torch.Size([4, 3, 8, 8]) torch.Size([3, 8, 8])
torch.Size([4, 3, 16, 16]) torch.Size([3, 16, 16])
torch.Size([4, 3, 6, 6]) torch.Size([3, 6, 6])
torch.Size([4, 3, 2, 2]) torch.Size([3, 2, 2])
torch.Size([4, 3, 4, 4]) torch.Size([3, 4, 4])
tensor([[[-0.1392, -0.4851],
         [ 0.6462,  0.5202]],

        [[ 0.9032,  0.0776],
         [ 0.4410,  0.1151]],

        [[ 1.0383,  0.1460],
         [ 0.0371,  0.1873]]]), tensor([[[ 0.8608,  0.5149],
         [ 1.6462,  1.5202]],

        [[-0.0484, -0.4612],
         [-0.2795, -0.4425]],

        [[ 0.3461,  0.0487],
         [ 0.0124,  0.0624]]])
torch.Size([3, 2, 2]) (2, 2)


D:\anaconda\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


当对图像依次进行多个变换操作时，可以使用torchvision.transforms.Compose类将这些变换连接在一起，构成一个统一的操作一次调用完成。示例如下：

In [32]:
from torchvision import transforms
from PIL import Image
orig_img =Image.open("a.png")
opts = transforms.Compose([
     transforms.CenterCrop(10),
     transforms.ToTensor(),
     transforms.Normalize((-100,-150,-200), (10,5,3)),
     transforms.ToPILImage()
])
new_img = opts(orig_img)
new_img.save("b.png")

以上介绍了torchvision提供的几个常见的模块，对于其他模块（如utils等）在此不再介绍，读者可自行阅读相关资料或官方文档。

## 5.4.4 torchaudio

Torchaudio是PyTorch提供的一个音频处理和识别的包，内置了很多针对音频文件的I/O操作、音频变换及特征提取、音频数据集、音频处理及自动语音识别（ASR，Automatic Speech Recognition）预训练模型等等。在使用torchaudio之前，先确保已经安装，可使用如下命令进行安装：

In [33]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchaudio
print(torchaudio.__version__)

0.12.1+cpu


下面的代码给出查看音频文件信息并进行显示的示例。

In [3]:
import requests  #导入从网络下载文件用到的包
import os
SAMPLE_WAV_URL ="https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SPEECH_FILE  = os.path.join(".", "speech.wav")
if not os.path.exists(SPEECH_FILE):  # 如果文件在本地不存在，则下载该文件
    with open(SPEECH_FILE, "wb") as file_:
        file_.write(requests.get(SAMPLE_WAV_URL).content)
        file_.close()
metadata = torchaudio.info(SPEECH_FILE)  # 得到音频文件信息
print(metadata)

AudioMetaData(sample_rate=16000, num_frames=54400, num_channels=1, bits_per_sample=16, encoding=PCM_S)


通过结果可知，该音频文件采样率为16000Hz，共有54400个采样，单声道，采样采用16bit编码，使用PCM_S编码方式。我们可以在当前目录中找到该文件并使用播放器进行播放，内容为一段3.4秒的男生英文语音：“I had that curiosity beside me at this moment. ”。

下面我们将使用预训练英文语音识别模型，对上例中的音频文件进行识别。代码如下：

In [6]:
import torch
# 得到可用的计算资源
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
# 打开音频文件
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)
# 设置预训练模型
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)
# 确定合适的采用频率
if sample_rate != bundle.sample_rate:
    Waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
# 对音频数据进行分类，得到每个音素对应的每个类别的概率
with torch.inference_mode():
    emission, _ = model(waveform)
# 定义针对分类结果的识别模型类    
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:      
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])
# 识别模型类实例化
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
# 针对分类结果进行识别，得到识别的字符串
transcript = decoder(emission[0])
# 打印最终的音频识别结果
print(transcript)

I|HAD|THAT|CURIOSITY|BESIDE|ME|AT|THIS|MOMENT|


## 5.4.5 模型持久化方法

与SKLearn的持久化类似，通过PyTorch训练好的模型往往需要部署到实际的业务系统中执行推理任务，这就需要对模型进行持久化，通常直接保存成文件即可。在实际的业务系统中使用时，再将模型文件加载到内存中使用。

PyTorch有两种模型保存的方式，一种是保存整个网络结构信息和模型参数信息，另一种方式是只保存网络的模型参数，不保存网络结构。

保存整个网络结构信息和模型参数信息的示例如下：


In [18]:
torch.save(GreedyCTCDecoder, './model.pth')

该代码将训练好的模型model_object保存到当前目录下的model.pth文件。以下代码完成模型加载，加载后即可使用该模型进行推理。

In [9]:
model = torch.load('./model.pth')

若仅保存网络的模型参数，则直接执行如下代码：

In [ ]:
torch.save(model_object.state_dict(), './params.pth')

加载模型时，需要先导入网络，然后再加载参数，示例如下：

In [ ]:
from models import VggModel
model = VggModel()
model.load_state_dict(torch.load('./params.pth'))

## 5.4.6 可视化工具包Visdom

Facebook专为Pytorch开发的实时可视化工具包Visdom，常用于实时显示训练过程的数据，具有灵活高效、界面美观等特点。安装Visdom非常简单，例如可在终端输入如下命令：

In [20]:
pip install visdom

Note: you may need to restart the kernel to use updated packages.


当我们在编写训练程序时，可以在代码中加入在Visdom进行数据展示的代码。这样，在执行训练程序时，便可在浏览器登陆后的界面中，查看可视化的效果，

下面代码演示Visdom根据数据点绘制曲线。

In [1]:
from visdom import Visdom
viz = Visdom()  # 初始化visdom类，默认可视化环境为main
viz.line([0.],  # Y的第一个点坐标
        [0.],    # X的第一个点坐标
        win="train loss",    #窗口名称
        opts=dict(title='train_loss')  # 图像标例
        )  #设置起始点
viz.line([1.],  # Y的下一个点坐标
        [1.],    # X的下一个点坐标
        win="train loss",  # 窗口名称，与上个窗口同名表示显示在同一个表格里
        update='append'    # 添加到上一个点后面
        ) 

Setting up a new session...


'train loss'

上述示例是在默认的main环境下展示，Visdom也支持在多环境下显示不同的可视化结果，如下代码展示了在设置的image环境下展示图像：

In [2]:
from visdom import Visdom
import numpy as np
image = np.random.randn(1, 3, 200, 200) # 一张3通道,200*200大小的图像
viz = Visdom(env='image') # 切换到image环境
viz.images(image, win='x')

Setting up a new session...


'x'

下面的程序演示了模拟loss损失下降的过程，并对其进行可视化。

In [ ]:
from visdom import Visdom
import numpy as np
import time

x = np.linspace(1, 100, 1000)  # 在区间[1,100]中等间隔取1000个样本
y = 1 / x  # 模拟损失的值
y = y.reshape(1000, 1)  # y变为shape为(1000,1)的二维数组
x = x.reshape(1000, 1)  # x变为shape为(1000,1)的二维数组

vis = Visdom(env='loss')  # 创建一个loss窗口
loss_window = vis.line(
    X=x[0],
    Y=y[0],
    opts={'xlabel': 'epochs', 'ylabel': 'value', 'title': 'loss'}  # 先声明窗口的标题和坐标轴的名称
)

for i in range(1, 1000):
    time.sleep(0.2) #模拟loss动态更新的效果
    vis.line(
        X=x[i],
        Y=y[i],
        win=loss_window,
        update='append'
    )

Setting up a new session...
